In [1]:
from data import EnformerDataset
from torch.utils.data import DataLoader

/home/buzun/filtered-transformer/.conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = EnformerDataset("/mnt/nfs_dna/DNALM/downstream_tasks/enformer/human/h5/human_train.h5")

In [3]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=12)

In [14]:
from transformers import AutoTokenizer
import numpy as np
seq, tg, coo = next(iter(train_loader))
tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base')
s =  "[CLS]".join([seq[0][i * 10: i * 10 + 10] for i in range(100)])
s


'GGTGAGAGCA[CLS]CCCATTGTTT[CLS]CTGCTGGTTA[CLS]CCCTCATGCG[CLS]AGATTTTTGT[CLS]TTGCTTCCTT[CLS]TCTCTCCCCA[CLS]GATACTTCTG[CLS]TTACTACTTC[CLS]ACTACTGTGG[CLS]GGAACTTCAC[CLS]AAAAATGCAT[CLS]CCCTCAGCAC[CLS]CCTGGGCTCT[CLS]GTGTCTGAAC[CLS]CACTTCACAT[CLS]GCCTTCTGTA[CLS]CTGCATCCCA[CLS]GCACTCCAGT[CLS]ATTCCTGAGC[CLS]TGAAATCTCA[CLS]GACCTCGTAA[CLS]TTACCAATCA[CLS]CTGTGCTTTT[CLS]GACACCCACC[CLS]CCTGCCACTC[CLS]ACTACCAGCT[CLS]TCTCTTATTT[CLS]TTCAACTTAT[CLS]TCTGGTTCAC[CLS]CAACGAAAAC[CLS]GATTCTTCAA[CLS]GCCCACTGGG[CLS]GCCTCAATCC[CLS]ATGTATTCAT[CLS]TATTGGCTGC[CLS]TCTCGTGAGG[CLS]TCAGATCTCT[CLS]TTTGTTTTTA[CLS]CTCATCTTAG[CLS]ATGTCATAGA[CLS]CTATCATGTG[CLS]CCCCTAGCTT[CLS]TCTTTGTTGT[CLS]ATTTGCCTTT[CLS]AAAGGTTTCA[CLS]ACCTTGCTCA[CLS]GATTTAACTC[CLS]TTCAACTGCT[CLS]CCCCACTTAT[CLS]TCCTGTGTGG[CLS]ATGAATGTGG[CLS]CTAGAGAAAA[CLS]ATGCACAAAC[CLS]ATGATGACAA[CLS]ATTTATGATC[CLS]ACAAATCCCA[CLS]AGTAAGCCTC[CLS]AACACCACTA[CLS]GGCCATTCTA[CLS]CTCTATGGCT[CLS]TTATTCAGTT[CLS]CCCAATGCCT[CLS]CATTCCACCC[CLS]CAAATTTTAA[CLS]GAAGATAATT[CLS]TCATCTCTT

In [19]:
import torch
s =  "[SEP]".join([seq[0][i * 10: i * 10 + 10] for i in range(100)])
tokens = torch.tensor(tokenizer(s)["input_ids"])
tokens[tokens == 1]


list

In [ ]:
import sys
sys.path.append("../..") 
from collections import namedtuple
from typing import Tuple, Optional
from torch import Tensor
from common_modules.transformers import BertRecurrentTransformerWithTokenizer
from memup.base import MemUpMemory, State, DataCollectorReplace, MemoryOut, DataCollectorAppend
from memup.loss import TS

DataType = namedtuple("DataType", ["text", "target", "length"])


class MemUpMemoryImpl(MemUpMemory[DataType]):

    def __init__(self, mem_tr: BertRecurrentTransformerWithTokenizer):
        super().__init__()
        self.mem_tr = mem_tr

    def forward(self, data: DataType, state: State) -> Tuple[Optional[Tensor], State]:
        os = self.mem_tr.forward(data.text, state)
        return None, os.state


class DataCollectorTrain(DataCollectorAppend[DataType, TS]):
    def apply(self, data: DataType, out: MemoryOut, state: State) -> TS:
        return TS(data.target, state)


class DataCollectorLastState(DataCollectorReplace[DataType, TS]):
    def apply(self, data: DataType, out: MemoryOut, state: State) -> TS:
        return TS(data.target, state)

In [ ]:
from torch import nn
from data_filters.sliding_window import SlidingWindowFilterTuple
from memup.loss import PredictorLoss, LossModule, PredictorLossStateOnly, EvalLossStateOnly
from memup.preproc import IncrementStep
from common_modules.transformers import BertRecurrentTransformerWithTokenizer, BertRecurrentTransformer, RecurrentTransformerFromBert, \
    BertRecurrentTransformerWithTokenizer
from transformers import AutoTokenizer
from transformers import BertModel, BertForSequenceClassification
from memup.base import MemoryRollout
import torch


rollout = 800
state_length = 50
torch.cuda.set_device("cuda:1")
data_filter = SlidingWindowFilterTuple[DataType](rollout, pad_fields={"text"}, padding=200, skip_fields={"target", "length"})


tokenizer = AutoTokenizer.from_pretrained('AIRI-Institute/gena-lm-bert-base')
bert: BertModel = BertModel.from_pretrained('AIRI-Institute/gena-lm-bert-base')
bert.train()
mem_transformer = BertRecurrentTransformerWithTokenizer(bert, tokenizer, 270, 4, 4, bert.config.hidden_size * 2).cuda()


class Predictor(nn.Module):

    def __init__(self):
        super().__init__()
        self.encoder = RecurrentTransformerFromBert(bert, 4, 4, bert.config.hidden_size * 2)
        self.head = nn.Sequential(
            nn.Linear(bert.config.hidden_size, bert.config.hidden_size),
            nn.ReLU(),
            nn.Linear(bert.config.hidden_size, bert.config.hidden_size),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(bert.config.hidden_size, 5313)
        )

    def forward(self, x, state):
        out = self.encoder.forward(x, state).out
        return self.head(out).abs()


predictor = Predictor().cuda()


opt = torch.optim.Adam([
    {"params": mem_transformer.bert.parameters(), "lr": 4e-6},
    {"params": predictor.parameters(), "lr": 2e-5},
    {"params": mem_transformer.encoder.parameters(), "lr": 2e-5},
])

memup_iter = MemoryRollout[DataType](
    steps=2,
    memory=MemUpMemoryImpl(mem_transformer),
    data_filter=data_filter,
    info_update=[IncrementStep()]
)




In [ ]:
text, target, coords = next(iter(train_loader))
state = torch.zeros(target.shape[0], state_length, bert.config.hidden_size, device=torch.device("cuda"))
T = len(text[0])
done = False
info = {}

while not done:
    data_collector, state, info, done = memup_iter.forward(DataType(text, target, T), state, info, DataCollectorTrain())
    print(info)